<a href="https://colab.research.google.com/github/sololzano/2021-Python-Optimization-Lab/blob/main/W7_Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic Algorithm

In [114]:
# Import libraries
from matplotlib import pyplot as plt
import numpy as np
import time

In [115]:
# Six Hump Camel Function
def binary_camel(x, encoding, x_min):
  # From binary encoding to real number
  new_x = np.array([[int(c, 2) for c in y] for y in x])
  X = (new_x * encoding) + x_min
  x1, x2 = X[:, 0], X[:, 1]
  
  # Same
  a = (4 - 2.1*x1**2 + (1/3)*x1**4)*x1**2
  b = x1*x2
  c = (-4 + 4*x2**2)*x2**2
  return a + b +c

In [116]:
# Adjiman's Benchmark Function 
def adjiman(x, encoding, x_min):
  # From binary encoding to real number
  new_x = np.array([[int(c, 2) for c in y] for y in x])
  X = (new_x * encoding) + x_min
  x1, x2 = X[:, 0], X[:, 1]
  a = np.cos(x1) * np.sin(x2)
  b = x1 / (x2**2 + 1)
  return (a - b)

In [127]:
def init_population(fitness_func, pop_size, dimensions, precision, x_min, x_max):
  x_decimal = np.random.randint(0, (2**precision) - 1, 
                                (pop_size, dimensions))
  # From base 10 to binary
  x_binary = [[format(j, '0{}b'.format(precision)) for j in i] for i in x_decimal]
  
  encoding = (x_max - x_min)/((2**precision) - 1)

  # Calculate fitness
  fitness = fitness_func(x_binary, encoding, x_min)

  # Get best 
  min_idx = np.argmin(fitness)
  gb = x_binary[min_idx]
  fgb = fitness[min_idx]

  return np.array(x_binary), np.array(fitness), np.array(gb), fgb, encoding

In [128]:
# fitness_func, pop_size, dimensions, precision, x_min, x_max
x_binary, fitness, gb, fgb, encoding = init_population(binary_camel, 10, 2, 10, -3, 3)
x_binary

array([['0001011101', '1100000011'],
       ['1110101111', '0010100000'],
       ['0010111100', '1000100011'],
       ['0011111110', '0110100000'],
       ['1111000100', '0101010000'],
       ['1001100111', '1111001100'],
       ['1011110100', '0111100010'],
       ['1111000010', '1011101011'],
       ['0010100010', '0001011110'],
       ['0011000101', '1110011010']], dtype='<U10')

In [129]:
def selection_mechanism(fitness, kind):
  if kind == 'random':
    indices = np.random.permutation(len(fitness))[:2]
    return indices
  if kind == 'tournament':
    indices = np.random.permutation(len(fitness))[:4]
    f_indices = fitness[indices]
    idx1 = np.argmin(f_indices[0:2])
    idx2 = np.argmin(f_indices[2:]) + 2
    indices = [indices[idx1], indices[idx2]]
    return np.array(indices)

In [131]:
indices = selection_mechanism(fitness, 'tournament')
parents = x_binary[indices]
print(indices)
print(parents)

[3 9]
[['0011111110' '0110100000']
 ['0011000101' '1110011010']]


In [170]:
def crossover(parents, pc, kind):
  p = np.random.uniform()
  if p > pc:
    return parents

  if kind == 'single_point':
    l = len(parents[0][0])
    idx = np.random.randint(l//4, l)
    p1 = parents[0]
    p2 = parents[1]
    children = []
    for x, y in zip(p1, p2):
      s1 = x[:idx] + y[idx:]
      s2 = y[:idx] + x[idx:]
      children.append([s1, s2])
    children = np.array(children).T
  elif kind == 'two_point':
    children = np.copy(parents)
    for _ in range(2):
      children = crossover(children, pc, 'single_point')
  return children

In [171]:
children = crossover(parents, 1., 'two_point')
print(np.array(parents), '\n')
print(children)

[['0011111110' '0110100000']
 ['0011000101' '1110011010']] 

[['0011000100' '0110011010']
 ['0011111111' '1110100000']]


In [ ]:
def mutation(child, pm, kind):
  if kind == 'bit_flip':
    new_child = []
    for i in range(len(child)):
      s = ''
      for j in range(len(child[i])):
        p = np.random.uniform()
        if p > pm:
          if child[i][j] == '1':
            s += '0'
          else:
            s += '1'
        else:
          s += child[i][j]
      new_child.append(s)
  if kind == 'substring_swap':
    pass
  return new_child

In [172]:
# Mutate children
print(children, '\n')
children[0] = mutation(children[0], 0.5, 'bit_flip')
children[1] = mutation(children[1], 0.5, 'bit_flip')
print(children)

[['0011000100' '0110011010']
 ['0011111111' '1110100000']] 

[['1000110110' '0101111010']
 ['1000101010' '0111000100']]


In [173]:
def maintenance_mechanism(parents, children, parents_fitness, 
                          children_fitness, kind):
  if kind == 'replacement':
    return children, children_fitness
  if kind == 'fittest':
    pooled_population = np.concatenate((parents, children))
    pooled_fitness = np.concatenate((parents_fitness, children_fitness))
    indices = np.argsort(pooled_fitness)[:len(pooled_fitness)//2]
    new_pop = pooled_population[indices]
    new_fit = pooled_fitness[indices]
    return new_pop, new_fit
  if kind == 'tournament':
    return 0

In [ ]:
def genetic_algorithm(fitness_function, pop_size, dimensions, precision, 
                      x_min, x_max, max_generations, selection_kind, 
                      crossover_kind, mutation_kind, maintenance_kind, pc, pm):
  
  # Initial population
  x, fx, gb, encoding = init_population(pop_size, dimensions, 
                                        precision, x_min, x_max)
  gb_array = []
  fb_array = []
  gb_array.append(gb)
  for i in range(1, max_generations + 1):
    new_generation = []
    new_fitness = []
    for j in range(pop_size // 2):
      # Get two potential parents
      idx_parents = selection_mechanism(fx, selection_kind)
      parents = np.copy(x[idx_parents])
      parents_fitness = np.copy(fitness_function[idx_parents])

      # Cross parents to generate two children
      children = crossover(parents, pc, crossover_kind)

      # Mutate children
      children, ch_fit = mutation(children, pm, mutation_kind)
      new_generation.append(np.array(children))
      new_fitness.append(np.array(ch_fit))

    # Re-shape new generation and respective fitness
    new_generation = np.reshape(new_generation, x.shape)
    new_fitness = np.reshape(new_fitness, fx.shape)

    # Maintenance mechanism
    x, fx = maintenance_mechanism(parents, new_generation, fx, 
                                  new_fitness, maintenance_kind)
    min_idx = np.argmin(fx)
    gb_array.append(x[min_idx])
    fb_array.append(fx[min_idx])
  return gb_array, fb_array